In [1]:
import jax
import jax.numpy as jnp
import numpy as np
import diffrax
import matplotlib.pyplot as plt

from soromox.systems.tendon_actuated_pcs import TendonActuatedPCS
from soromox.systems.system_state import SystemState

from controllers.MPPI import ModelPredictivePathIntegralControl

from diffrax import diffeqsolve, ODETerm, Euler, Tsit5, SaveAt

In [2]:
num_segments = 2  # number of segments in the robot
params = {
    "p0": jnp.array([jnp.pi / 2, jnp.pi / 2, 0.0, 0.0, 0.0, 0.0]),  # Initial position and orientation
    "L": 1e-1 * jnp.ones((num_segments,)),  # default: 1e-1
    "r": 2e-2 * jnp.ones((num_segments,)),  # default: 2e-2
    "rho": 1070 * jnp.ones((num_segments,)),  # Volumetric density of Dragon Skin 20 [kg/m^3]
    "g": jnp.array([0.0, 0.0, 9.81]),  # Gravity vector [m/s^2]
    "E": 2e3 * jnp.ones((num_segments,)),  # Elastic modulus [Pa]
    "G": 1e3 * jnp.ones((num_segments,)),  # Shear modulus [Pa]
}
params["D"] = 1e-3 * jnp.diag((jnp.repeat(jnp.array([[1e0, 1e0, 1e0, 1e3, 1e3, 1e3]]), num_segments, axis=0) * params["L"][:, None]).flatten())
active_tendon_routing_params = {
    "ry": 2e-2 * jnp.array([1.0, -1.0]),  # y-coordinate of the pulling point of the tendons [m]
    "rz": 2e-2 * jnp.array([0.0, 0.0]),  # z-coordinate of the pulling point of the tendons [m]
    "my": jnp.array([0.0, 0.0]),  # slope coefficient in the x-y plane of the tendons [-]
    "mz": jnp.array([0.0, 0.0]),  # slope coefficient in the x-z plane of the tendons [-]
    "idx_seg_att": jnp.array([1, 0]),  # length of the tendons = x-coordinate of the attachment points [m]
}
passive_tendon_routing_params = {
    "ry": 2e-2 * jnp.array([0.0]),  # y-coordinate of the pulling point of the tendons [m]
    "rz": 2e-2 * jnp.array([1.0]),  # z-coordinate of the pulling point of the tendons [m]
    "my": jnp.array([0.0]),  # slope coefficient in the x-y plane of the tendons [-]
    "mz": jnp.array([0.0]),  # slope coefficient in the x-z plane of the tendons [-]
    "idx_seg_att": jnp.array([1]),  # length of the tendons = x-coordinate of the attachment points [m]
}
passive_tendon_params = {
    "k_pt": jnp.array([0.6]),
    "d_pt": jnp.array([0.1]),
    "l_pt0": jnp.array([-0.3]),
}

# ======================================================
# Robot initialization
# ======================================================
robot = TendonActuatedPCS(
    num_segments=num_segments,
    params=params,
    active_tendon_routing_params=active_tendon_routing_params,
    passive_tendon_routing_params=passive_tendon_routing_params,
    passive_tendon_params=passive_tendon_params
)


nx = 2 * 6 * num_segments
nu = 2

dynamics_model = lambda t, x, u: robot.forward_dynamics(t, x, (u,))

In [3]:
N = 30
dt = 0.01
Q = jnp.diag(jnp.array([100.0]*int(nx/2) + [1.0]*int(nx/2)))
R = jnp.diag(jnp.array([0.01]*nu))

# chi_eq = jnp.array([[-jnp.pi/2, 0.0, -robot.L.sum()]])
q_eq = jnp.zeros(nx//2)
qd_eq = jnp.zeros_like(q_eq)
x_eq = jnp.concatenate([q_eq, qd_eq])
u_eq = jnp.zeros(nu)

q0 = jnp.repeat(jnp.array([0.0, 0.0, 5.0 * jnp.pi, 0.1, 0.2, 0.0])[None, :], num_segments, axis=0).flatten()
qd0 = jnp.zeros_like(q0)
x0 = jnp.concatenate([q0, qd0])

controller = ModelPredictivePathIntegralControl(dynamics_model, N, dt, nu, Q, R, x_eq, u_max=2.0, sigma=0.00001, correlation=0.9)

u_test = controller.control_input[:, 0]

In [4]:
initial_state = SystemState(0.0, x0)

robot.rollout_to(initial_state, u_test, t1=30e-3)

SystemState(t=Array([0.  , 0.01, 0.02, 0.03], dtype=float64), y=Array([[ 0.00000000e+00,  0.00000000e+00,  1.57079633e+01,
         1.00000000e-01,  2.00000000e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.57079633e+01,
         1.00000000e-01,  2.00000000e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.93420640e-02, -2.99153795e-02,  1.56911666e+01,
         1.00639434e-01,  1.98129675e-01, -4.65046031e-04,
         1.18316059e-02, -5.26377237e-02,  1.56874954e+01,
         9.60865594e-02,  1.97527602e-01, -1.76049230e-04,
        -2.07075164e+00, -3.61647969e+00, -6.29754784e+00,
         4.76525194e-02, -3.09950275e-01, -5.18431271e-02,
         8.89088327e-01, -5.55996454e+00, -7.19619846e-01,
        -5.02335127e-01, -2.57075367e-01, -1.64893

In [5]:
print(diffeqsolve(ODETerm(dynamics_model), Euler(), 0.0, N*dt, 1e-4, x0, u_test, saveat=SaveAt(ts=(dt * jnp.arange(N+1)))).ys)

[[ 0.00000000e+00  0.00000000e+00  1.57079633e+01  1.00000000e-01
   2.00000000e-01  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.57079633e+01  1.00000000e-01  2.00000000e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.93400006e-02 -2.98883906e-02  1.56916275e+01  1.00641183e-01
   1.98141604e-01 -4.65530167e-04  1.18419224e-02 -5.26470316e-02
   1.56873972e+01  9.60913616e-02  1.97527556e-01 -1.75926008e-04
  -2.07145060e+00 -3.61910202e+00 -6.30074786e+00  4.75137376e-02
  -3.10479036e-01 -5.18381597e-02  8.87664096e-01 -5.55956335e+00
  -7.07416859e-01 -5.02739993e-01 -2.56971625e-01 -1.65003286e-02]
 [-4.00626755e-02 -6.80795350e-02  1.55839503e+01  1.01007634e-01
   1.94034720e-01 -9.43448080e-04  2.00148166e-02 -1.07574601e-01
   1.56865131e+01  9.07015464e-02  1.94930018e-01 -3.48718751e-04
  -2.057

In [6]:
t0 = 0.0
solver_dt = 2e-4
T = N*dt

term = ODETerm(lambda t, x, args: controller.dynamics(t, x, jnp.zeros(controller.nu)))
saveat = SaveAt(ts=(t0 + controller.dt * jnp.arange(controller.N+1)))
print(diffeqsolve(terms=term, solver=Euler(), t0=t0, t1=t0+controller.T+solver_dt, dt0=solver_dt, y0=x0, saveat=saveat).ys)

ValueError: Terms are not compatible with solver! Got:
ODETerm(vector_field=<function <lambda>>)
but expected:
diffrax.AbstractTerm
Note that terms are checked recursively: if you scroll up you may find a root-cause error that is more specific.

In [ ]:
controller.compute_trajectory_cost(controller.control_input, x0)

JitTracer<float64[31,24,1]>


Array([[1035847.31378697]], dtype=float64)